# v1 training

In [ ]:
import os
import sys
import random
import warnings
import string
import torch.nn as nn

from pathlib import Path

import torch as torch
from IPython.display import SVG, display

from torch.utils.data import DataLoader

### Enable Hot Reload

In [ ]:
%load_ext autoreload
%autoreload 2

### Edit Python path
Add the `models` directory to Python's `path`.

In [ ]:
b_paths = [os.path.abspath(os.path.join('..', '..', '..')), os.path.abspath(os.path.join('..', '..')), os.path.abspath(os.path.join('..', '..', 'scripts'))]
for b_path in b_paths:
    if b_path not in sys.path:
        sys.path.append(b_path)

BASE_DIR = Path(os.getcwd()).parent.parent.parent.resolve()
%cd $BASE_DIR

### Ignore Warnings

In [ ]:
warnings.filterwarnings('ignore')

### Import Helpers

In [ ]:
from models.scripts.transformer.Transformer import Transformer
from models.scripts.transformer.utils import strokes_to_svg, build_vocab, preprocess_dataset, seed_all, initialize_weights, pad_collate_fn, tensor_to_word
from models.scripts.generate_dataset import WordDatasetGenerator, WordGenerator

### Configuration Settings

In [ ]:
VERSION = "vt1"
SEED = 2021
BATCH_SIZE = 256

In [ ]:
seed_all(SEED) # Reproducibility

### Create Vocabulary

In [ ]:
CHARS = list(string.ascii_letters)
BLANK_SPACE = [" "]

In [ ]:
VOCAB = build_vocab(CHARS+BLANK_SPACE)

N_TOKENS = len(VOCAB)
PAD_IDX = VOCAB['<pad>']
BOS_IDX = VOCAB['<bos>']
EOS_IDX = VOCAB['<eos>']

print(f"Number of Tokens: {N_TOKENS}\n")
print(VOCAB.freqs, "\n") # Distribution
print({VOCAB.itos[i]: i for i in range(N_TOKENS)}) # Token order

### Generate Datasets

In [ ]:
use_cache = True

if use_cache: # Generate from cache file
    d_gen = WordDatasetGenerator(vocab = VOCAB, fname="words_stroke_100_155805")
    train, valid, test = d_gen.generate_from_cache()

else: # Generate from scratch and cache (if regenerated, results could change)
    news_commentary_path = os.path.join(BASE_DIR, "word_sources", "news-commentary-v14.en")
    words = WordGenerator().generate_from_file(news_commentary_path)
    BRUSH_SPLIT=0.15
    d_gen = WordDatasetGenerator(vocab = VOCAB, words=words[:int(len(words)*(1-BRUSH_SPLIT))], extended_dataset=False)
    d_gen.generate()
    d_gen.add_training_words(words[int(len(words)*(1-BRUSH_SPLIT)):])
    train, valid, test = d_gen.generate_from_cache()

### Create Dataset for PyTorch

In [ ]:
train_set = DataLoader(preprocess_dataset(train, VOCAB,  os.path.join(d_gen.fname, "train.pt"), total_len=d_gen.get_learning_set_length("train")), batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate_fn)
valid_set = DataLoader(preprocess_dataset(valid, VOCAB,  os.path.join(d_gen.fname, "valid.pt"), total_len=d_gen.get_learning_set_length("valid")), batch_size=BATCH_SIZE, shuffle=False, collate_fn=pad_collate_fn)

### Inspect Generated Data

In [ ]:
# Get random index
x_dummy, y_dummy = next(iter(valid_set)) # Create dummy for visualization
ind = random.choice(range(y_dummy.shape[0]))
print("Index:", ind)

print()
print("X Shape:", x_dummy[ind].shape)
# Show actual expr for first tensor
print("Y Shape:", y_dummy[ind].shape)
print("Label:", tensor_to_word(y_dummy[ind], VOCAB))

# Get length of subplot depending on granularity (exclude bos/eos for strokes)
svg_str = strokes_to_svg(x_dummy[ind], {'height':100, 'width':100}, d_gen.padding_value, BOS_IDX, EOS_IDX)
display(SVG(data = svg_str))


print()
print(f'X[{ind}]:', x_dummy[ind])
print()

eos_tensor = torch.zeros(x_dummy[ind].size(-1)) + EOS_IDX


for i, row in enumerate(x_dummy[ind]):
    if torch.all(row.eq(eos_tensor)):
        print("EOS is in position:", i)
        break

In [ ]:
vec_num=0
print(f'X[{ind}][{vec_num}]:', x_dummy[ind][vec_num])
svg_str = strokes_to_svg(x_dummy[ind][vec_num].unsqueeze(0), {'height':100, 'width':100}, d_gen.padding_value, BOS_IDX, EOS_IDX)
display(SVG(data = svg_str))

### Model Hyper-parameters/Create Transformer Model

In [ ]:
model= Transformer(VERSION, VOCAB, n_tokens=N_TOKENS, encoder_name='v74-bpe', decoder_name='new')
model.decoder.apply(initialize_weights)
model.save_hyperparameters_to_json()
model.count_parameters()
print(f"Encoder trainable parameters: {sum(p.numel() for p in model.encoder.parameters() if p.requires_grad):,}.")
print(f"Decoder trainable parameters: {sum(p.numel() for p in model.decoder.parameters() if p.requires_grad):,}.")
print("\n\n\n", model)
model.to(model.device)

### Training process along with best-model caching

In [ ]:
LEARNING_RATE = 5e-4
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=300, gamma=0.5)

In [ ]:
model.train_loop(resume=False,
                 train_set=train_set,
                 valid_set=valid_set,
                 optimizer=optimizer,
                 scheduler=scheduler,
                 n_epochs=4000)

### Plot Training  Logs

In [ ]:
model.plot_training()